In [1]:
!pip install pytorch-forecasting

     |████████████████████████████████| 113 kB 18.8 MB/s 
     |████████████████████████████████| 527 kB 41.6 MB/s 
     |████████████████████████████████| 308 kB 45.1 MB/s 
     |████████████████████████████████| 210 kB 50.0 MB/s 
     |████████████████████████████████| 81 kB 4.3 MB/s 
     |████████████████████████████████| 134 kB 43.4 MB/s 
     |████████████████████████████████| 397 kB 23.5 MB/s 
     |████████████████████████████████| 829 kB 30.6 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 952 kB 50.8 MB/s 
     |████████████████████████████████| 1.1 MB 49.9 MB/s 
     |████████████████████████████████| 271 kB 52.1 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 53.0 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
     |████████████████████████████████| 150 kB 50.4 MB/s 
     |████████████████████████████████| 49 kB 4.9 MB/s 
     |████████████████

In [4]:
import torch
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from time import time 
from pylab import rcParams
from tqdm import tqdm
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim
from pytorch_forecasting.metrics import SMAPE
from tensorflow.keras.utils import to_categorical

In [11]:
one = pd.read_csv('1등_쓰레기봉투.csv')
two = pd.read_csv("2등_경유.csv")
three = pd.read_csv("3등_보건용마스크(공적판매용).csv")
four = pd.read_csv("4등_콘크리트벽돌.csv")
five = pd.read_csv("5등_라벨용지.csv")
six = pd.read_csv("6등_보건용마스크.csv")
seven = pd.read_csv("7등_미장벽돌.csv")
eight = pd.read_csv("8등_기타화초.csv") 
nine = pd.read_csv("9등_강관파일.csv")
ten = pd.read_csv("10등_음식물쓰레기처리통.csv")
eleven = pd.read_csv("11등_레미콘.csv")
twelve = pd.read_csv("12등_금속제창.csv")

In [53]:
# one['납품요구접수일자'] =  pd.to_datetime(one['납품요구접수일자'])
# one = one.set_index('납품요구접수일자')
# one = one.set_index('납품요구접수일자')

In [15]:
#월, 년, 주 구하기

one['month'] = one.index.month
one['year'] = one.index.year
one['dayofweek'] = one.index.dayofweek

In [20]:
#예측 정확도 체크를 위해 smape 값 구하기
def smape(pred, target, values = False):
  if values : # value 여러개인 list
    v = (np.abs(pred - target)) / (np.abs(pred)+np.abs(target))
    v = np.mean(v)
    
  else: #단일 value
    v = torch.abs(pred - target) / (torch.abs(pred) + torch.abs(target))

  return v*100

#train model 만들어서 underfitting, overfitting 확인
def train_model(model, train_data, train_labels, val_data=None, val_labels=None, num_epochs=100, verbose = 10):
  
  optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
  train_hist = []
  val_hist = []
  for t in range(num_epochs):
    epoch_loss = 0
    for idx, seq in enumerate(train_data): # sample 별 hidden state reset을 해줘야 함 
        model.reset_hidden_state()
        
        y_pred = model(seq)
        target = torch.tensor(train_labels[idx])
        
        loss = smape(y_pred[0].float(), target)
        
        # loss = loss_fn.loss(y_pred[0].float(), train_labels[idx]) # 1개의 step에 대한 loss

        # update weights
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        epoch_loss += loss.item()

    train_hist.append(epoch_loss / len(train_data))
    
    if val_data is not None:
      with torch.no_grad():
        val_loss = 0
        val_preds=np.array([])
        val_targets=np.array([])
        for val_idx, val_seq in enumerate(val_data):
            model.reset_hidden_state() #seq 별로 hidden state 초기화 
            y_val_pred = model(val_seq)
            val_target = torch.tensor(val_labels[val_idx])
            
            val_preds = np.append(val_preds, np.array([y_val_pred]))
            val_targets = np.append(val_targets, np.array([val_target]))

        val_smape = smape(val_preds, val_targets, values=True)

        val_hist.append(val_smape) # val hist에 추가
          
      if t % verbose == 0: # epoch loss print
          print(f'Epoch {t} train loss: {(epoch_loss / len(train_data)):.3f}%, val loss: {val_smape:.3f}%')
            
  return model, train_hist, val_hist, val_preds, val_targets
  
#RNN 모델 구현하기
class BTCPredictor1(nn.Module):
  def __init__(self, n_features, n_hidden, seq_len, n_layers):
    super(BTCPredictor1, self).__init__()
    self.n_hidden = n_hidden 
    self.seq_len = seq_len 
    self.n_layers = n_layers 
    self.rnn = nn.RNN(n_features, n_hidden, n_layers, batch_first = True)
    self.fc = nn.Sequential(nn.Linear(n_hidden * seq_len, out_features=1), nn.Sigmoid()
    )

  def reset_hidden_state(self):
    self.hidden_layer = (
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden),
        torch.zeros(self.n_layers, self.seq_len, self.n_hidden)
    )

  def forward(self, seq): # sequences: 1 X time X features
    seq = torch.unsqueeze(seq, 0) # 3차원으로 만듦
    ho = torch.zeros(self.n_layers, seq.size()[0], self.n_hidden) #seq : [x1~x30] : sequence     Times X Features
    out, _ = self.rnn(seq, ho) # out: RNN의 마지막 레이어로부터 나온 output feature 를 반환한다. hn: hidden state를 반환한다.
    out = out.reshape(out.shape[0], -1) # many to many 전략
    out = self.fc(out)
    return out


 

In [55]:


#x와 y값 sequence형태로 만들기
def create_sequences(one, seq_length):
    xs = []
    ys = []
    for i in range(one.shape[0]-seq_length):
        x = one.iloc[i:(i+seq_length)]
        y = one.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(one, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(one)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=one.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 51.507%, val loss: 49.919%
Epoch 10 train loss: 49.924%, val loss: 49.822%
Epoch 20 train loss: 49.924%, val loss: 49.821%
Epoch 30 train loss: 49.924%, val loss: 49.821%
Epoch 40 train loss: 49.924%, val loss: 49.821%
Epoch 50 train loss: 49.924%, val loss: 49.821%
Epoch 60 train loss: 49.924%, val loss: 49.821%
Epoch 70 train loss: 49.924%, val loss: 49.821%
Epoch 80 train loss: 49.924%, val loss: 49.821%
Epoch 90 train loss: 49.924%, val loss: 49.821%


In [56]:
from sklearn.metrics import mean_squared_error as MSE
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

RMSE :  2.09


In [57]:
# two['납품요구접수일자'] =  pd.to_datetime(two['납품요구접수일자'])
# two = two.set_index('납품요구접수일자')
# two = two.set_index('납품요구접수일자')

In [58]:
#월, 년, 주 구하기

two['month'] = two.index.month
two['year'] = two.index.year
two['dayofweek'] = two.index.dayofweek

In [59]:


#x와 y값 sequence형태로 만들기
def create_sequences(two, seq_length):
    xs = []
    ys = []
    for i in range(two.shape[0]-seq_length):
        x = two.iloc[i:(i+seq_length)]
        y = two.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(two, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(two)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=two.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 54.682%, val loss: 49.808%
Epoch 10 train loss: 49.619%, val loss: 48.913%
Epoch 20 train loss: 49.506%, val loss: 48.905%
Epoch 30 train loss: 49.458%, val loss: 48.903%
Epoch 40 train loss: 49.434%, val loss: 48.902%
Epoch 50 train loss: 49.421%, val loss: 48.902%
Epoch 60 train loss: 49.414%, val loss: 48.902%
Epoch 70 train loss: 49.411%, val loss: 48.902%
Epoch 80 train loss: 49.409%, val loss: 48.901%
Epoch 90 train loss: 49.407%, val loss: 48.901%


In [60]:
from sklearn.metrics import mean_squared_error as MSE
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

RMSE :  1.80


In [61]:
# three['납품요구접수일자'] =  pd.to_datetime(three['납품요구접수일자'])
# three = three.set_index('납품요구접수일자')
# three = three.set_index('납품요구접수일자')

#월, 년, 주 구하기

three['month'] = three.index.month
three['year'] = three.index.year
three['dayofweek'] = three.index.dayofweek

In [62]:


#x와 y값 sequence형태로 만들기
def create_sequences(three, seq_length):
    xs = []
    ys = []
    for i in range(three.shape[0]-seq_length):
        x = three.iloc[i:(i+seq_length)]
        y = three.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(three, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(three)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=three.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 61.333%, val loss: 73.122%
Epoch 10 train loss: 51.870%, val loss: 70.631%
Epoch 20 train loss: 51.799%, val loss: 70.425%
Epoch 30 train loss: 51.781%, val loss: 70.335%
Epoch 40 train loss: 51.774%, val loss: 70.283%
Epoch 50 train loss: 51.770%, val loss: 70.249%
Epoch 60 train loss: 51.769%, val loss: 70.225%
Epoch 70 train loss: 51.767%, val loss: 70.209%
Epoch 80 train loss: 51.767%, val loss: 70.197%
Epoch 90 train loss: 51.766%, val loss: 70.190%


In [63]:
from sklearn.metrics import mean_squared_error as MSE
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

RMSE :  1.73


In [64]:
# four['납품요구접수일자'] =  pd.to_datetime(four['납품요구접수일자'])
# four = four.set_index('납품요구접수일자')
# four = four.set_index('납품요구접수일자')

#월, 년, 주 구하기

four['month'] = four.index.month
four['year'] = four.index.year
four['dayofweek'] = four.index.dayofweek

In [65]:


#x와 y값 sequence형태로 만들기
def create_sequences(four, seq_length):
    xs = []
    ys = []
    for i in range(four.shape[0]-seq_length):
        x = four.iloc[i:(i+seq_length)]
        y = four.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(four, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(four)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=four.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 52.749%, val loss: 49.843%
Epoch 10 train loss: 49.203%, val loss: 48.721%
Epoch 20 train loss: 49.111%, val loss: 48.653%
Epoch 30 train loss: 49.085%, val loss: 48.633%
Epoch 40 train loss: 49.075%, val loss: 48.625%
Epoch 50 train loss: 49.072%, val loss: 48.621%
Epoch 60 train loss: 49.071%, val loss: 48.620%
Epoch 70 train loss: 49.070%, val loss: 48.619%
Epoch 80 train loss: 49.070%, val loss: 48.619%
Epoch 90 train loss: 49.070%, val loss: 48.618%


In [66]:
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

RMSE :  1.81


In [67]:
# five['납품요구접수일자'] =  pd.to_datetime(five['납품요구접수일자'])
# five = five.set_index('납품요구접수일자')
# five = five.set_index('납품요구접수일자')

#월, 년, 주 구하기

five['month'] = five.index.month
five['year'] = five.index.year
five['dayofweek'] = five.index.dayofweek

In [68]:


#x와 y값 sequence형태로 만들기
def create_sequences(five, seq_length):
    xs = []
    ys = []
    for i in range(five.shape[0]-seq_length):
        x = five.iloc[i:(i+seq_length)]
        y = five.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(five, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(five)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=five.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 73.142%, val loss: 64.118%
Epoch 10 train loss: 53.428%, val loss: 44.058%
Epoch 20 train loss: 52.391%, val loss: 43.263%
Epoch 30 train loss: 52.136%, val loss: 43.072%
Epoch 40 train loss: 52.033%, val loss: 42.999%
Epoch 50 train loss: 51.981%, val loss: 42.964%
Epoch 60 train loss: 51.951%, val loss: 42.946%
Epoch 70 train loss: 51.933%, val loss: 42.935%
Epoch 80 train loss: 51.922%, val loss: 42.928%
Epoch 90 train loss: 51.915%, val loss: 42.924%


In [69]:
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

RMSE :  1.70


In [70]:
# six['납품요구접수일자'] =  pd.to_datetime(six['납품요구접수일자'])
# six = six.set_index('납품요구접수일자')
# six = six.set_index('납품요구접수일자')

#월, 년, 주 구하기

six['month'] = six.index.month
six['year'] = six.index.year
six['dayofweek'] = six.index.dayofweek

In [71]:


#x와 y값 sequence형태로 만들기
def create_sequences(six, seq_length):
    xs = []
    ys = []
    for i in range(six.shape[0]-seq_length):
        x = six.iloc[i:(i+seq_length)]
        y = six.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(six, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(six)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=six.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 52.099%, val loss: 52.735%
Epoch 10 train loss: 50.116%, val loss: 52.459%
Epoch 20 train loss: 50.055%, val loss: 52.419%
Epoch 30 train loss: 50.044%, val loss: 52.410%
Epoch 40 train loss: 50.042%, val loss: 52.409%
Epoch 50 train loss: 50.041%, val loss: 52.408%
Epoch 60 train loss: 50.041%, val loss: 52.408%
Epoch 70 train loss: 50.041%, val loss: 52.408%
Epoch 80 train loss: 50.041%, val loss: 52.408%
Epoch 90 train loss: 50.041%, val loss: 52.408%


In [72]:
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

RMSE :  2.52


In [51]:
# seven['납품요구접수일자'] =  pd.to_datetime(seven['납품요구접수일자'])
# seven = seven.set_index('납품요구접수일자')
# seven = seven.set_index('납품요구접수일자')

#월, 년, 주 구하기

seven['month'] = seven.index.month
seven['year'] = seven.index.year
seven['dayofweek'] = seven.index.dayofweek

In [52]:


#x와 y값 sequence형태로 만들기
def create_sequences(seven, seq_length):
    xs = []
    ys = []
    for i in range(seven.shape[0]-seq_length):
        x = seven.iloc[i:(i+seq_length)]
        y = seven.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(seven, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(seven)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=seven.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 50.708%, val loss: 49.604%
Epoch 10 train loss: 49.091%, val loss: 49.290%
Epoch 20 train loss: 49.090%, val loss: 49.277%
Epoch 30 train loss: 49.089%, val loss: 49.274%
Epoch 40 train loss: 49.089%, val loss: 49.274%
Epoch 50 train loss: 49.089%, val loss: 49.274%
Epoch 60 train loss: 49.089%, val loss: 49.273%
Epoch 70 train loss: 49.089%, val loss: 49.273%
Epoch 80 train loss: 49.089%, val loss: 49.273%
Epoch 90 train loss: 49.089%, val loss: 49.273%


In [54]:
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

RMSE :  1.99


In [75]:
# eight['납품요구접수일자'] =  pd.to_datetime(eight['납품요구접수일자'])
# eight = eight.set_index('납품요구접수일자')
# eight = eight.set_index('납품요구접수일자')

#월, 년, 주 구하기

eight['month'] = eight.index.month
eight['year'] = eight.index.year
eight['dayofweek'] = eight.index.dayofweek

In [76]:


#x와 y값 sequence형태로 만들기
def create_sequences(eight, seq_length):
    xs = []
    ys = []
    for i in range(eight.shape[0]-seq_length):
        x = eight.iloc[i:(i+seq_length)]
        y = eight.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(eight, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(eight)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=eight.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 51.726%, val loss: 49.958%
Epoch 10 train loss: 50.373%, val loss: 49.801%
Epoch 20 train loss: 50.372%, val loss: 49.801%
Epoch 30 train loss: 50.372%, val loss: 49.801%
Epoch 40 train loss: 50.372%, val loss: 49.801%
Epoch 50 train loss: 50.372%, val loss: 49.801%
Epoch 60 train loss: 50.372%, val loss: 49.801%
Epoch 70 train loss: 50.372%, val loss: 49.801%
Epoch 80 train loss: 50.372%, val loss: 49.801%
Epoch 90 train loss: 50.372%, val loss: 49.801%
RMSE :  2.16


In [81]:
# nine['납품요구접수일자'] =  pd.to_datetime(nine['납품요구접수일자'])
# nine = nine.set_index('납품요구접수일자')
# nine = nine.set_index('납품요구접수일자')

#월, 년, 주 구하기

nine['month'] = nine.index.month
nine['year'] = nine.index.year
nine['dayofweek'] = nine.index.dayofweek

In [82]:


#x와 y값 sequence형태로 만들기
def create_sequences(nine, seq_length):
    xs = []
    ys = []
    for i in range(nine.shape[0]-seq_length):
        x = nine.iloc[i:(i+seq_length)]
        y = nine.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(nine, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(nine)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=nine.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 52.259%, val loss: 50.131%
Epoch 10 train loss: 49.250%, val loss: 48.839%
Epoch 20 train loss: 49.180%, val loss: 48.741%
Epoch 30 train loss: 49.167%, val loss: 48.699%
Epoch 40 train loss: 49.165%, val loss: 48.678%
Epoch 50 train loss: 49.164%, val loss: 48.668%
Epoch 60 train loss: 49.164%, val loss: 48.663%
Epoch 70 train loss: 49.164%, val loss: 48.661%
Epoch 80 train loss: 49.164%, val loss: 48.659%
Epoch 90 train loss: 49.164%, val loss: 48.659%
RMSE :  1.94


In [86]:
# ten['납품요구접수일자'] =  pd.to_datetime(ten['납품요구접수일자'])
# ten = ten.set_index('납품요구접수일자')
# ten = ten.set_index('납품요구접수일자')

#월, 년, 주 구하기

ten['month'] = ten.index.month
ten['year'] = ten.index.year
ten['dayofweek'] = ten.index.dayofweek

In [87]:


#x와 y값 sequence형태로 만들기
def create_sequences(ten, seq_length):
    xs = []
    ys = []
    for i in range(ten.shape[0]-seq_length):
        x = ten.iloc[i:(i+seq_length)]
        y = ten.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(ten, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(ten)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=ten.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 50.309%, val loss: 49.966%
Epoch 10 train loss: 49.353%, val loss: 49.884%
Epoch 20 train loss: 49.353%, val loss: 49.884%
Epoch 30 train loss: 49.353%, val loss: 49.884%
Epoch 40 train loss: 49.353%, val loss: 49.884%
Epoch 50 train loss: 49.353%, val loss: 49.884%
Epoch 60 train loss: 49.353%, val loss: 49.884%
Epoch 70 train loss: 49.353%, val loss: 49.884%
Epoch 80 train loss: 49.353%, val loss: 49.884%
Epoch 90 train loss: 49.353%, val loss: 49.884%
RMSE :  2.15


In [92]:
# eleven['납품요구접수일자'] =  pd.to_datetime(eleven['납품요구접수일자'])
# eleven = eleven.set_index('납품요구접수일자')
# eleven = eleven.set_index('납품요구접수일자')

#월, 년, 주 구하기

eleven['month'] = eleven.index.month
eleven['year'] = eleven.index.year
eleven['dayofweek'] = eleven.index.dayofweek

In [93]:


#x와 y값 sequence형태로 만들기
def create_sequences(eleven, seq_length):
    xs = []
    ys = []
    for i in range(eleven.shape[0]-seq_length):
        x = eleven.iloc[i:(i+seq_length)]
        y = eleven.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(eleven, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(eleven)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=eleven.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 53.237%, val loss: 53.254%
Epoch 10 train loss: 51.928%, val loss: 53.144%
Epoch 20 train loss: 51.927%, val loss: 53.144%
Epoch 30 train loss: 51.927%, val loss: 53.144%
Epoch 40 train loss: 51.927%, val loss: 53.144%
Epoch 50 train loss: 51.927%, val loss: 53.144%
Epoch 60 train loss: 51.927%, val loss: 53.144%
Epoch 70 train loss: 51.927%, val loss: 53.144%
Epoch 80 train loss: 51.927%, val loss: 53.144%
Epoch 90 train loss: 51.927%, val loss: 53.144%
RMSE :  2.70


In [96]:
#twelve['납품요구접수일자'] =  pd.to_datetime(twelve['납품요구접수일자'])
#twelve = twelve.set_index('납품요구접수일자')
#twelve = twelve.set_index('납품요구접수일자')

#월, 년, 주 구하기

twelve['month'] = twelve.index.month
twelve['year'] = twelve.index.year
twelve['dayofweek'] = twelve.index.dayofweek

In [97]:


#x와 y값 sequence형태로 만들기
def create_sequences(twelve, seq_length):
    xs = []
    ys = []
    for i in range(twelve.shape[0]-seq_length):
        x = twelve.iloc[i:(i+seq_length)]
        y = twelve.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, Y = create_sequences(twelve, seq_length)


# 학습과 테스트 데이터 분리
split_ratio = 0.8
split = int(len(twelve)*split_ratio)

# split
X_train = X[:split]
X_val = X[split:]
Y_train = Y[:split]
Y_val = Y[split:]

# to tensor
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).float()
Y_val = torch.from_numpy(Y_val).float()

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_val, Y_val)

batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


model = BTCPredictor1(n_features=twelve.shape[1],
                   n_hidden=4,
                   seq_len=10,
                   n_layers=1,
                   )

#underfitting, overfitting 확인
model, train_hist, val_hist, val_preds, val_targets = train_model(
    model,
    X_train,
    Y_train,
    X_val,
    Y_val,
    num_epochs=100,
    verbose=10)
 
# rmse(root mean square error)
rmse = np.sqrt(MSE(val_targets, val_preds))
print("RMSE : % .2f" %(rmse))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with 

Epoch 0 train loss: 51.932%, val loss: 51.586%
Epoch 10 train loss: 50.937%, val loss: 51.513%
Epoch 20 train loss: 50.937%, val loss: 51.513%
Epoch 30 train loss: 50.937%, val loss: 51.513%
Epoch 40 train loss: 50.937%, val loss: 51.513%
Epoch 50 train loss: 50.937%, val loss: 51.513%
Epoch 60 train loss: 50.937%, val loss: 51.513%
Epoch 70 train loss: 50.937%, val loss: 51.513%
Epoch 80 train loss: 50.937%, val loss: 51.513%
Epoch 90 train loss: 50.937%, val loss: 51.513%
RMSE :  2.45
